# Tâche #1 : Classification d'incidents avec un réseau *feedforward* et des *embeddings* Spacy

On reprend la classification des descriptions d’accidents du premier travail. Le corpus de textes contient 3 partitions : 
-	un fichier d’entraînement -  data/incidents_train.json
-	un fichier de validation -  data/incidents_dev.json
-	un fichier de test - data/incidents_test.json

Entraînez un modèle de réseau de neurones de type feedforward multicouche (MLP) avec plongements de mots pour déterminer le type d’un incident à partir de sa description. 

Voici les consignes pour cette tâche : 

-	Nom du notebook : mlp.ipynb
-	Tokenisation : Utilisation de Spacy. 
-	Plongements de mots : Ceux de Spacy. 
-	Normalisation : Aucune normalisation. 
-	Agrégation des plongements de mots : Comparer les approches max, average et min pooling. 
-	Structure du réseau : 1 seule couche cachée dont vous choisirez la taille (à expliquer). 
-	Présentez clairement vos résultats et faites-en l’analyse. En cas de doute, inspirez-vous de ce qui a été fait dans le travail pratique #1. 

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [115]:
import spacy

spacy_model = spacy.load("en_core_web_md")

In [116]:
import pandas as pd
import json
import numpy as np
# Assurez-vous que le modèle de langue de spacy est téléchargé
# python -m spacy download fr_core_news_md (par exemple pour le français)

# Charger le modèle de langue de spacy

# Définition des chemins vers les fichiers de données
train_data_path = './data/incidents_train.json'
dev_data_path = './data/incidents_dev.json'
test_data_path = './data/incidents_test.json'

def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)
        
        text = [item["text"] for item in incident_list]
        target = np.array([int(item["label"]) for item in incident_list])
         
    return text, target


# Créer les DataFrames pour chaque partition de données
train_list, train_target = load_incident_dataset(train_data_path)
dev_list, dev_target = load_incident_dataset(dev_data_path)
test_list, test_target = load_incident_dataset(test_data_path)

# Affichage de l'information de base sur les DataFrames
display(f"Train data: text_size {len(train_list)}, target_size {len(train_target)}")
display(f"Dev data: text_size {len(dev_list)}, target_size {len(dev_target)}")
display(f"Test data: text_size {len(test_list)}, target_size {len(test_target)}")



# Vérification des premiers enregistrements dans l'ensemble d'entraînement
train_list[0]


'Train data: text_size 2475, target_size 2475'

'Dev data: text_size 531, target_size 531'

'Test data: text_size 531, target_size 531'

' At approximately 8:50 a.m. on October 29  1997  Employee #1 was painting a  single story house at 2657 7th Ave  Sacramento  CA. He was caulking around the  peak of the roof line on the west side of the house  20 ft above the ground.  He was working off of a 24 ft aluminum extension ladder so that his feet were  approximately 12 to 13 feet above the ground. Employee #1 fell and suffered a  concussion and two dislocated discs in his lower back and was hospitalized.  The ladder was not secured to prevent movement.                                 '

Ce script utilise les bibliothèques pandas, json et numpy pour manipuler les données. Il charge les données d'incidents à partir de fichiers JSON, les organise dans des ensembles de textes et d'étiquettes, crée des DataFrames pour chaque ensemble de données, affiche des informations de base sur ces ensembles, puis vérifie le premier enregistrement dans l'ensemble d'entraînement. 

## 2. Gestion de plongements de mots (*embeddings*)

In [117]:
# word_embeddings = {}

# for train in train_list:
#     doc = spacy_model(train['text'])
#     for token in doc:
#         word_embeddings[token.text] = token.vector

# nb_dim = 50
# word = "painting"
# print("\nLes {} premières valeurs du vecteur de plongement du mot \"{}\": \n{}...".format(nb_dim, word, word_embeddings[word][:nb_dim]))

In [118]:
docs = []
for train in train_list:
    docs.append(spacy_model(train))

In [119]:
import numpy as np
# Fonction pour calculer l'embedding moyen d'une phrase
def average_embedding(sentence, nlp_model=spacy_model):
    tokenised_sentence = nlp_model(sentence)      # Tokenisation de la phrase
    nb_column = len(tokenised_sentence)     # Nombre de colonnes dans la matrice d'embedding (nombre de tokens)
    nb_rows =  nlp_model.vocab.vectors_length  # Nombre de lignes dans la matrice d'embedding (taille de l'espace d'embedding)
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))                                  
    for index, token in enumerate(tokenised_sentence):  # Remplissage de la matrice d'embedding avec les vecteurs d'embedding de chaque token
        sentence_embedding_matrix[:, index] = token.vector
    return np.average(sentence_embedding_matrix, axis=1)  # Calcul de l'embedding moyen en prenant la moyenne le long de l'axe des colonnes

# Fonction pour calculer l'embedding avec la technique du max pooling d'une phrase
def maxpool_embedding(sentence, nlp_model=spacy_model): 
    tokenised_sentence = nlp_model(sentence)
    nb_column = len(tokenised_sentence)
    nb_rows =  nlp_model.vocab.vectors_length 
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))                                    
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    return np.max(sentence_embedding_matrix, axis=1)

# Fonction pour calculer l'embedding avec la technique du min pooling d'une phrase
def minpool_embedding(sentence, nlp_model=spacy_model):
    tokenised_sentence = nlp_model(sentence)
    nb_column = len(tokenised_sentence)
    nb_rows =  nlp_model.vocab.vectors_length 
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))                                    
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    return np.min(sentence_embedding_matrix, axis=1)

Ces fonctions prennent une phrase en entrée, la tokenisent à l'aide d'un modèle spaCy (nlp_model), puis calculent l'embedding moyen, l'embedding avec la technique du max pooling, ou l'embedding avec la technique du min pooling en utilisant les vecteurs d'embedding des tokens.

In [120]:
display(average_embedding(train_list[0]))

array([-1.96873228e+00, -5.10729270e-01, -2.35349378e+00,  4.97672888e-01,
        4.88353263e+00, -5.49114923e-01,  5.77439719e-01,  4.37476939e+00,
        9.82608505e-01,  2.76181211e-01,  3.32291929e+00,  1.83452836e+00,
       -2.19539340e+00,  7.13141577e-01,  4.67925282e-01,  1.69569208e+00,
       -4.15554943e-01,  5.46220097e-01, -8.17133067e-01, -1.81754243e+00,
        1.45045962e+00, -1.64859988e-01,  7.35189274e-01,  1.39185669e+00,
       -2.16275255e-01, -1.53836771e+00, -2.90574922e+00, -1.10088644e+00,
       -2.45176188e-01,  1.23890522e+00,  7.16629948e-01,  2.70870438e-01,
       -4.62952329e-01, -2.58849184e+00, -2.63352788e+00, -1.38033428e+00,
       -6.32679274e-01,  1.24110528e+00,  4.78200293e-01,  2.11488465e-02,
        1.16837779e+00, -2.70511874e-01, -1.43659251e-01,  9.54270908e-01,
       -1.01153907e+00,  1.14068031e+00,  1.59027040e-01, -2.24387173e-01,
       -2.60219337e-03,  1.40369611e+00, -4.18182354e-01,  1.69740604e+00,
       -1.48766517e+00, -

## 3. Création de modèle(s)

In [121]:
from torch import nn


class MultiLayerPerceptron(nn.Module):
    # Initialisation du modèle avec les paramètres d'entrée, de la couche cachée et de sortie
    def __init__(self, input_size, hidden_layer_size, output_size) :
        super().__init__()
        self.intput_layer = nn.Linear(input_size, hidden_layer_size)    # Définition de la couche d'entrée (input layer) avec une transformation linéaire
        self.output_layer = nn.Linear(hidden_layer_size, output_size) # Définition de la couche de sortie (output layer) avec une transformation linéaire
        
    def forward(self, x):
        x = self.intput_layer(x)
        x = nn.functional.relu_(x)
        x = self.output_layer(x)
        return x

Cette classe MultiLayerPerceptron hérite de la classe "nn.Module" de PyTorch et définit un modèle de perceptron multicouche avec une couche d'entrée, une couche cachée avec une fonction d'activation ReLU, et une couche de sortie. La méthode "forward" spécifie comment les données passent à travers le modèle lors de l'inférence.

In [122]:
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor
from typing import List

class SpacyDataset(Dataset):
    # Initialisation de la classe avec les données du dataset, les cibles, et une fonction d'agrégation des phrases
    def __init__(self, dataset: List[str] , target: np.array, sentence_aggregation_function):
        self.dataset = dataset
        self.doc_embeddings = [None for _ in range(len(dataset))]  # Initialisation d'une liste vide pour stocker les embeddings des phrases (spaCy)
        self.sentence_aggregation_function = sentence_aggregation_function # Stockage de la fonction d'agrégation des phrases
        self.target = target
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        if self.doc_embeddings[index] is None:
            self.doc_embeddings[index] = self.sentence_aggregation_function(self.dataset[index])  
        return FloatTensor(self.doc_embeddings[index]), LongTensor([self.target[index]]).squeeze(0)

In [156]:
# Un dictionnaire pour choisir le type d'agrégation
aggregation = {
    "average" : average_embedding,
    "maxpool" : maxpool_embedding,
    "minpool" : minpool_embedding
}

# On détermine ici comment la classe SpacyDataset construit la représentation d'un texte
# par l'agrégation des représentations de mots. 
# Choix possibles: "average", "maxpool", "minpool"
aggregation_function = aggregation["average"]  

# On finalise la construction des 3 jeux de données et leurs dataloaders
train_dataset = SpacyDataset(train_list, train_target, aggregation_function)
valid_dataset = SpacyDataset(dev_list, dev_target, aggregation_function)
test_dataset = SpacyDataset(test_list, test_target, aggregation_function)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

Ce code utilise un dictionnaire appelé 'aggregation' pour associer des chaînes de caractères à des fonctions d'agrégation, telles que 'average_embedding, maxpool_embedding, et minpool_embedding'. Il choisit ensuite une fonction d'agrégation spécifique (dans cet exemple, "average") pour être utilisée dans la construction des jeux de données (train_dataset, valid_dataset, test_dataset) en utilisant la classe SpacyDataset, et leurs DataLoader respectifs (train_dataloader, valid_dataloader, test_dataloader). Les DataLoader sont utilisés pour faciliter l'itération sur les jeux de données pendant l'entraînement et l'évaluation du modèle.

In [135]:
len(train_dataset[0])

2

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

## 5. Entraînement de modèle(s)

In [157]:
embedding_size = spacy_model.meta['vectors']['width'] # La dimension des vecteurs d'embeddings de Spacy
nb_classes = len([i for i in range(9)])

print("Taille des plongements de Spacy:", embedding_size)
print("Nombre de classes:", nb_classes)

Taille des plongements de Spacy: 300
Nombre de classes: 9


Ce code utilise les métadonnées du modèle spaCy pour obtenir la dimension des vecteurs d'embedding (embedding_size). Ensuite, il définit le nombre de classes (nb_classes) en comptant le nombre d'entiers dans la plage de 0 à 8. Enfin, il affiche ces informations. Notez que le nombre de classes dans cet exemple est fixé à 9

In [186]:
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
import numpy as np

set_seeds(42) # Fixation des graines pour assurer la reproductibilité
hidden_size = 100 # Définition de la taille de la couche cachée dans le modèle MLP
optimizer = "SGD"

directory_name = 'model/{}_mlp_optimizer{}'.format(aggregation_function.__name__, optimizer)  

# Création du modèle MLP
model = MultiLayerPerceptron(embedding_size, hidden_size, nb_classes)

# Création d'une instance de la classe Experiment de Poutyne pour gérer l'entraînement et l'évaluation
experiment = Experiment(directory_name, 
                        model, 
                        optimizer = optimizer, 
                        task="classification")
model

MultiLayerPerceptron(
  (intput_layer): Linear(in_features=300, out_features=100, bias=True)
  (output_layer): Linear(in_features=100, out_features=9, bias=True)
)

Ce code utilise la bibliothèque Poutyne pour définir une expérience d'entraînement. Il crée un modèle MLP avec les paramètres spécifiés, puis utilise la classe 'Experiment' de Poutyne pour gérer l'entraînement et l'évaluation du modèle. Le répertoire spécifié (directory_name) sera utilisé pour sauvegarder le modèle et les résultats de l'expérienc

In [187]:

logging = experiment.train(train_dataloader, valid_dataloader, epochs=50, disable_tensorboard=True)

Epoch:  1/50 Train steps: 155 Val steps: 34 0.21s loss: 1.683572 acc: 40.363636 fscore_macro: 0.094113 val_loss: 1.606964 val_acc: 40.677966 val_fscore_macro: 0.081529
Epoch 1: val_acc improved from -inf to 40.67797, saving file to model/average_embedding_mlp_optimizerSGD/checkpoint_epoch_1.ckpt
Epoch:  2/50 Train steps: 155 Val steps: 34 0.21s loss: 1.549883 acc: 44.080808 fscore_macro: 0.142711 val_loss: 1.504527 val_acc: 50.847458 val_fscore_macro: 0.191354
Epoch 2: val_acc improved from 40.67797 to 50.84746, saving file to model/average_embedding_mlp_optimizerSGD/checkpoint_epoch_2.ckpt
Epoch:  3/50 Train steps: 155 Val steps: 34 0.26s loss: 1.443486 acc: 48.848485 fscore_macro: 0.183284 val_loss: 1.437503 val_acc: 49.905838 val_fscore_macro: 0.193641
Epoch:  4/50 Train steps: 155 Val steps: 34 0.21s loss: 1.377818 acc: 52.121212 fscore_macro: 0.209421 val_loss: 1.319665 val_acc: 51.977401 val_fscore_macro: 0.191257
Epoch 4: val_acc improved from 50.84746 to 51.97740, saving file t

## 6. Évaluation et analyse de résultats

In [185]:
# Évaluation du modèle sur l'ensemble de test à l'aide du DataLoader test_dataloader
experiment.test(test_dataloader)

Found best checkpoint at epoch: 38
lr: 0.001, loss: 0.581567, acc: 79.5152, fscore_macro: 0.799569, val_loss: 0.92777, val_acc: 70.9981, val_fscore_macro: 0.548251
Loading checkpoint model/average_embedding_mlp_optimizerADAM/checkpoint_epoch_38.ckpt
Running test
Test steps: 34 0.03s test_loss: 0.930146 test_acc: 68.926554 test_fscore_macro: 0.636161    


{'time': 0.026028541033156216,
 'test_loss': 0.930146022252528,
 'test_acc': 68.92655367231639,
 'test_fscore_macro': 0.6361610293388367}